#  추천 시스템 - 데이터베이스 저장 및 Text2SQL 실행

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

## ETF 데이터 로드

- ETF 목록: CSV 다운로드 ( http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC020103010901 )
- ETF 상세 정보: 이전 단계에서 수집해서 저장한 CSV 문서를 로드

`(1) ETF 목록`

In [3]:
import pandas as pd
import numpy as np

# ETF 목록
etf_data = pd.read_csv('data/etf_list.csv', encoding='cp949')
etf_data.head()

,종목코드,종목명,상장일,분류체계,운용사,수익률(최근 1년),기초지수,추적오차,순자산총액,괴리율,변동성,복제방법,총보수,과세유형
0,466400,1Q 25-08 회사채(A+이상)액티브,2023/09/19,채권-회사채-단기,하나자산운용,4.52,KIS 2025-08만기형 크레딧 A+이상 지수(총수익),0.11,111916276404,0.03,매우낮음,실물(액티브),0.10,배당소득세(보유기간과세)
1,491610,1Q CD금리액티브(합성),2024/09/24,기타,하나자산운용,0.00,KIS 하나 CD금리 총수익지수,0.05,316206006696,0.02,매우낮음,합성(액티브),0.02,배당소득세(보유기간과세)
2,451060,1Q K200액티브,2023/01/31,주식-시장대표,하나자산운용,-3.66,코스피 200,0.77,99754348820,-0.01,높음,실물(액티브),0.18,배당소득세(보유기간과세)
3,463290,1Q 단기금융채액티브,2023/08/03,채권-혼합-단기,하나자산운용,4.01,MK 머니마켓 지수(총수익),0.05,252717462257,0.00,매우낮음,실물(액티브),0.08,배당소득세(보유기간과세)
4,479080,1Q 머니마켓액티브,2024/04/02,채권-혼합-단기,하나자산운용,0.00,KIS-하나 MMF 지수(총수익),0.06,308255065986,-0.01,매우낮음,실물(액티브),0.05,배당소득세(보유기간과세)


`(2) ETF 상세 정보`

In [4]:
from glob import glob
exsiting_csv_files = glob('data/etf_info/etf_info_*.csv')

len(exsiting_csv_files)

914

In [5]:
import pandas as pd

df_list = []    
for file in exsiting_csv_files:
    df = pd.read_csv(file)
    df = df.set_index('항목').T
    
    # 필요한 컬럼만 선택 (표준코드 제외, URL 중복 컬럼 제외)
    required_columns = ['한글명', '영문명', '종목코드', '상장일', '펀드형태', '기초지수명', '추적배수', '자산운용사',
                       '지정참가회사(AP)', '총보수(%)', '회계기간', '과세유형', '분배금 지급일', '홈페이지',
                       '기초 시장', '기초 자산', '기본 정보', '투자유의사항']
    
    # 존재하는 컬럼만 선택
    available_columns = [col for col in required_columns if col in df.columns]
    df = df[available_columns]
    
    df_list.append(df)

etf_info = pd.concat(df_list)
etf_info = etf_info.dropna(axis=1, how='all')

print(f"etf_info.shape: {etf_info.shape}")
etf_info.head()

etf_info.shape: (914, 18)


항목,한글명,영문명,종목코드,상장일,펀드형태,기초지수명,추적배수,자산운용사,지정참가회사(AP),총보수(%),회계기간,과세유형,분배금 지급일,홈페이지,기초 시장,기초 자산,기본 정보,투자유의사항
내용,TIGER 엔비디아미국채커버드콜밸런스(합성),TIGER SYNTH-NVIDIA UST Cvd Call Balanced,0000D0,2024-12-17,수익증권형,KEDI 엔비디아미국채30년타겟커버드콜혼합지수(TR),일반 (1),미래에셋자산운용(주),"미래, 키움, 메리츠, 삼성",0.39,"매년 1월 1일부터 12월 31일까지. 단, 최초 회계기간은 최초설정일로부터 당해연...",배당소득세(보유기간과세),매월 마지막 영업일,www.tigeretf.com,(해외) (북미) (미국),(혼합자산) (주식+채권) / (-) (-) / (-),"- 이 ETF는 주로 채권 및 채권관련 집합투자증권을을 주된 투자대상자산으로 하며,...",- 이 ETF의 수익률은 보수 또는 비용 등 이 ETF의 순자산가치에 부의 영향을 ...
내용,KODEX 200,KODEX 200,069500,2002-10-14,수익증권형,코스피 200,일반 (1),삼성자산운용(주),NaN,0.15,매년 1월 1일부터 12월 31일까지,비과세,"매 1, 4, 7, 10월의 마지막 영업일 및 회계기간 종료일(종료일이 영업일이 아...",http://www.kodex.com,(국내) (코스피) (-),(주식) (시장대표) / (-) (-) / (-),1좌당 순자산가치의 변동률을 기초지수인 KOSPI200의 변동률과 유사하도록 투자신...,"규정에서 정하는 한도를 초과하여 추적오차가 발생하고 일정기간 이상 지속될 경우, 지..."
내용,KOSEF 200,KOSEF 200,069660,2002-10-14,수익증권형,코스피 200,일반 (1),키움투자자산운용(주),NaN,0.05,매년 1월 1일부터 12월 31일까지,비과세,"매 1, 4, 7, 10월의 마지막 영업일 및 회계기간 종료일(종료일이 영업일이 아...",http://www.kosef.co.kr,(국내) (코스피) (-),(주식) (시장대표) / (-) (-) / (-),1좌당 순자산가치의 변동률을 기초지수인 KOSPI 200 지수의 변동률과 유사하도록...,"이 투자신탁은 기초지수와 동일한 수익률을 실현함을 그 투자목적으로 하고 있으나, 투..."
내용,KODEX 반도체,KODEX SEMICON,091160,2006-06-27,수익증권형,KRX 반도체,일반 (1),삼성자산운용(주),NaN,0.45,매년 1월 1일부터 12월 31일까지.,비과세,"매 1, 4, 7, 10월의 마지막 영업일 및 회계기간 종료일(종료일이 영업일이 아...",http://www.kodex.com,(국내) (코스피+코스닥) (-),(주식) (업종섹터) / (-) (정보기술) / (-),1좌당 순자산가치의 변동률을 기초지수인 KRX Semicon 지수의 변동률과 유사하...,특정 산업에 속하는 주식에 투자하므로 국내 주식시장 전체의 성과와는 크게 다를 수 ...
내용,KODEX 은행,KODEX BANKs,091170,2006-06-27,수익증권형,KRX 은행,일반 (1),삼성자산운용(주),NaN,0.3,매년 1월 1일부터 12월 31일까지.,비과세,"매 1, 4, 7, 10월의 마지막 영업일 및 회계기간 종료일(종료일이 영업일이 아...",http://www.kodex.com,(국내) (코스피+코스닥) (-),(주식) (업종섹터) / (-) (금융) / (-),1좌당 순자산가치의 변동률을 기초지수인 KRX Banks 지수의 변동률과 유사하도록...,특정 산업에 속하는 주식에 투자하므로 국내 주식시장 전체의 성과와는 크게 다를 수 ...


## **Text2SQL** 

- 자연어로 된 질문을 **SQL 쿼리**로 자동 변환

- 데이터베이스 스키마 기반으로 **정확한 쿼리** 생성

- 개발자가 아닌 사용자도 **데이터베이스 검색** 가능

- 데이터베이스 접근성을 높이는 자연어 인터페이스 기술

- 실습 데이터: ETF 목록 데이터 ('`data/etf_list.csv`')


### 1) **SQLite Database**에 저장

- ETF 데이터를 CSV에서 읽어 SQLite **데이터베이스 테이블** 생성

- **데이터 타입** 최적화: 종목코드(INTEGER PRIMARY KEY), 수익률/총보수(FLOAT), 문자열(TEXT)

- 기본 **통계 분석**: 전체 ETF 수, 운용사 수, 평균 수익률/총보수 등 산출

In [6]:
# ETF 목록
etf_data = pd.read_csv('data/etf_list.csv', encoding='cp949')
etf_data.head(2)

,종목코드,종목명,상장일,분류체계,운용사,수익률(최근 1년),기초지수,추적오차,순자산총액,괴리율,변동성,복제방법,총보수,과세유형
0,466400,1Q 25-08 회사채(A+이상)액티브,2023/09/19,채권-회사채-단기,하나자산운용,4.52,KIS 2025-08만기형 크레딧 A+이상 지수(총수익),0.11,111916276404,0.03,매우낮음,실물(액티브),0.10,배당소득세(보유기간과세)
1,491610,1Q CD금리액티브(합성),2024/09/24,기타,하나자산운용,0.00,KIS 하나 CD금리 총수익지수,0.05,316206006696,0.02,매우낮음,합성(액티브),0.02,배당소득세(보유기간과세)


In [7]:
# 열 이름 변경
etf_data.columns = ['종목코드', '종목명', '상장일', '분류체계', '운용사', '수익률_최근1년', '기초지수', '추적오차',
       '순자산총액', '괴리율', '변동성', '복제방법', '총보수', '과세유형']

In [8]:
import pandas as pd

# 데이터 타입 변환
def convert_to_numeric_safely(value):
    try:
        return pd.to_numeric(value)
    except:
        return None

# 각 컬럼의 데이터 타입 변환
etf_data['종목코드'] = etf_data['종목코드'].apply(lambda x: str(x).strip())
etf_data['수익률_최근1년'] = etf_data['수익률_최근1년'].apply(convert_to_numeric_safely)
etf_data['추적오차'] = etf_data['추적오차'].apply(convert_to_numeric_safely)
etf_data['순자산총액'] = etf_data['순자산총액'].apply(convert_to_numeric_safely)
etf_data['괴리율'] = etf_data['괴리율'].apply(convert_to_numeric_safely)
etf_data['총보수'] = etf_data['총보수'].apply(convert_to_numeric_safely)

# 문자열 데이터 정리
string_columns = ['종목명', '상장일', '분류체계', '운용사', '기초지수', '변동성', '복제방법', '과세유형']
for col in string_columns:
    etf_data[col] = etf_data[col].astype(str).apply(lambda x: x.strip())


# 데이터 타입 확인
etf_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   종목코드      930 non-null    object 
 1   종목명       930 non-null    object 
 2   상장일       930 non-null    object 
 3   분류체계      930 non-null    object 
 4   운용사       930 non-null    object 
 5   수익률_최근1년  930 non-null    float64
 6   기초지수      930 non-null    object 
 7   추적오차      930 non-null    float64
 8   순자산총액     930 non-null    int64  
 9   괴리율       930 non-null    float64
 10  변동성       930 non-null    object 
 11  복제방법      930 non-null    object 
 12  총보수       930 non-null    float64
 13  과세유형      930 non-null    object 
dtypes: float64(4), int64(1), object(9)
memory usage: 101.8+ KB


In [9]:
import pandas as pd
import sqlite3

# SQLite 데이터베이스 생성
conn = sqlite3.connect('etf_database.db')
cursor = conn.cursor()

# 테이블 삭제 (if exists)
cursor.execute("DROP TABLE IF EXISTS ETFs")

# 테이블 생성
cursor.execute("""
CREATE TABLE ETFs (
    종목코드 TEXT PRIMARY KEY,
    종목명 TEXT,
    상장일 TEXT,
    분류체계 TEXT,
    운용사 TEXT,
    수익률_최근1년 REAL,
    기초지수 TEXT,
    추적오차 REAL,
    순자산총액 REAL,
    괴리율 REAL,
    변동성 TEXT,
    복제방법 TEXT,
    총보수 REAL,
    과세유형 TEXT
)
""")

# 데이터 삽입
for _, row in etf_data.iterrows():
    try:
        cursor.execute("""
        INSERT INTO ETFs VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            str(row['종목코드']),
            str(row['종목명']),
            str(row['상장일']),
            str(row['분류체계']),
            str(row['운용사']),
            float(row['수익률_최근1년']) if pd.notna(row['수익률_최근1년']) else None,
            str(row['기초지수']),
            float(row['추적오차']) if pd.notna(row['추적오차']) else None,
            float(row['순자산총액']) if pd.notna(row['순자산총액']) else None,
            float(row['괴리율']) if pd.notna(row['괴리율']) else None,
            str(row['변동성']),
            str(row['복제방법']),
            float(row['총보수']) if pd.notna(row['총보수']) else None,
            str(row['과세유형'])
        ))
    except Exception as e:
        print(f"Error inserting row: {row}")
        print(f"Error message: {str(e)}")
        continue

# 변경사항 저장
conn.commit()

# 데이터베이스 상태 확인
cursor.execute("SELECT COUNT(*) FROM ETFs")
etf_count = cursor.fetchone()[0]
print(f"\n=== 데이터베이스 생성 완료 ===")
print(f"ETF 개수: {etf_count}")


=== 데이터베이스 생성 완료 ===
ETF 개수: 930


In [10]:
# 데이터 확인
print("\n=== 데이터 샘플 ===")
cursor.execute("SELECT * FROM ETFs LIMIT 1")
columns = [description[0] for description in cursor.description]
row = cursor.fetchone()
if row:
    for col, val in zip(columns, row):
        print(f"{col}: {val}")


=== 데이터 샘플 ===
종목코드: 466400
종목명: 1Q 25-08 회사채(A+이상)액티브
상장일: 2023/09/19
분류체계: 채권-회사채-단기
운용사: 하나자산운용
수익률_최근1년: 4.52
기초지수: KIS 2025-08만기형 크레딧 A+이상 지수(총수익)
추적오차: 0.11
순자산총액: 111916276404.0
괴리율: 0.03
변동성: 매우낮음
복제방법: 실물(액티브)
총보수: 0.1
과세유형: 배당소득세(보유기간과세)


In [11]:
# 데이터베이스 종료
conn.close()

### 2) **LangChain**에 연동

- **LangChain**과 ETF DB 연동으로 자연어 쿼리 처리 가능

- **GPT**와 **Gemini** 모델을 활용한 SQL 쿼리 자동 생성

- 한국어 응답을 위한 **QA Chain** 구성 및 쿼리 실행 도구 설정

`(1) DB 스키마 확인`
   - 작업의 첫 단계로 테이블 목록 확인 필요
   - 각 테이블의 **구조와 관계** 파악을 위한 스키마 정보 검토

In [12]:
from langchain_community.utilities import SQLDatabase

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri("sqlite:///etf_database.db")

# 사용 가능한 테이블 목록 출력
tables = db.get_usable_table_names()
print(tables)

['ETFs', 'ETFsInfo']


In [13]:
# 테이블 스키마 정보 출력
print(db.get_table_info())


CREATE TABLE "ETFs" (
	"종목코드" TEXT, 
	"종목명" TEXT, 
	"상장일" TEXT, 
	"분류체계" TEXT, 
	"운용사" TEXT, 
	"수익률_최근1년" REAL, 
	"기초지수" TEXT, 
	"추적오차" REAL, 
	"순자산총액" REAL, 
	"괴리율" REAL, 
	"변동성" TEXT, 
	"복제방법" TEXT, 
	"총보수" REAL, 
	"과세유형" TEXT, 
	PRIMARY KEY ("종목코드")
)

/*
3 rows from ETFs table:
종목코드	종목명	상장일	분류체계	운용사	수익률_최근1년	기초지수	추적오차	순자산총액	괴리율	변동성	복제방법	총보수	과세유형
466400	1Q 25-08 회사채(A+이상)액티브	2023/09/19	채권-회사채-단기	하나자산운용	4.52	KIS 2025-08만기형 크레딧 A+이상 지수(총수익)	0.11	111916276404.0	0.03	매우낮음	실물(액티브)	0.1	배당소득세(보유기간과세)
491610	1Q CD금리액티브(합성)	2024/09/24	기타	하나자산운용	0.0	KIS 하나 CD금리 총수익지수	0.05	316206006696.0	0.02	매우낮음	합성(액티브)	0.02	배당소득세(보유기간과세)
451060	1Q K200액티브	2023/01/31	주식-시장대표	하나자산운용	-3.66	코스피 200	0.77	99754348820.0	-0.01	높음	실물(액티브)	0.18	배당소득세(보유기간과세)
*/


CREATE TABLE "ETFsInfo" (
	"한글명" TEXT, 
	"영문명" TEXT, 
	"종목코드" TEXT, 
	"상장일" TEXT, 
	"펀드형태" TEXT, 
	"기초지수명" TEXT, 
	"추적배수" TEXT, 
	"자산운용사" TEXT, 
	"지정참가회사" TEXT, 
	"총보수" TEXT, 
	"회계기간" TEXT, 
	"과세유형" TEXT, 
	"분배금지급일" TEXT, 
	"홈페이지" TEXT, 
	"기초시장" TEXT, 
	"기초

In [14]:
# 기본 쿼리 실행
query = "SELECT * FROM ETFs LIMIT 5"
result = db.run(query)
print(result)

[('466400', '1Q 25-08 회사채(A+이상)액티브', '2023/09/19', '채권-회사채-단기', '하나자산운용', 4.52, 'KIS 2025-08만기형 크레딧 A+이상 지수(총수익)', 0.11, 111916276404.0, 0.03, '매우낮음', '실물(액티브)', 0.1, '배당소득세(보유기간과세)'), ('491610', '1Q CD금리액티브(합성)', '2024/09/24', '기타', '하나자산운용', 0.0, 'KIS 하나 CD금리 총수익지수', 0.05, 316206006696.0, 0.02, '매우낮음', '합성(액티브)', 0.02, '배당소득세(보유기간과세)'), ('451060', '1Q K200액티브', '2023/01/31', '주식-시장대표', '하나자산운용', -3.66, '코스피 200', 0.77, 99754348820.0, -0.01, '높음', '실물(액티브)', 0.18, '배당소득세(보유기간과세)'), ('463290', '1Q 단기금융채액티브', '2023/08/03', '채권-혼합-단기', '하나자산운용', 4.01, 'MK 머니마켓 지수(총수익)', 0.05, 252717462257.0, 0.0, '매우낮음', '실물(액티브)', 0.08, '배당소득세(보유기간과세)'), ('479080', '1Q 머니마켓액티브', '2024/04/02', '채권-혼합-단기', '하나자산운용', 0.0, 'KIS-하나 MMF 지수(총수익)', 0.06, 308255065986.0, -0.01, '매우낮음', '실물(액티브)', 0.05, '배당소득세(보유기간과세)')]


`(2) SQL Chain 설정`
- **SQL Chain** 구성을 위한 **GPT**와 **Gemini** 모델 통합 설정
- 정규식을 활용한 **SQL 쿼리 자동 추출** 기능 구현
- 모델 간 **연계 프로세스** 최적화로 쿼리 생성 효율 향상
- AI 모델 기반 SQL 쿼리 자동화 가능

In [15]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_sql_query_chain

# SQL Chain 설정
gpt_llm = ChatOpenAI(model="gpt-4o-mini")
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

gpt_sql = create_sql_query_chain(llm=gpt_llm, db=db)
gemini_sql = create_sql_query_chain(llm=gemini_llm, db=db)

# 쿼리 실행
test_question = "상위 5개 운용사별 ETF 개수는 몇 개인가요?"

gpt_generated_sql = gpt_sql.invoke({'question':test_question})
gemini_generated_sql = gemini_sql.invoke({'question':test_question})

print(f"Answer (GPT):\n{gpt_generated_sql}")
print("-"*100)
print(f"Answer (Gemini):\n{gemini_generated_sql}")
print("-"*100)

h:\miniconda3\envs\modu2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Answer (GPT):
```sql
SELECT "운용사", COUNT(*) AS "ETF_개수"
FROM "ETFs"
GROUP BY "운용사"
ORDER BY "ETF_개수" DESC
LIMIT 5;
```
----------------------------------------------------------------------------------------------------
Answer (Gemini):
```sqlite
SELECT "운용사", COUNT(*) AS "ETF_개수" FROM ETFs GROUP BY "운용사" ORDER BY "ETF_개수" DESC LIMIT 5
```
----------------------------------------------------------------------------------------------------


In [16]:
import re
from typing import Optional

def extract_sql(text: str) -> Optional[str]:
    """
    주어진 텍스트에서 SQL 쿼리를 추출하고 정리
    
    Parameters:
        text (str): SQL 쿼리가 포함된 텍스트
        
    Returns:
        Optional[str]: 정리된 SQL 쿼리 또는 None
        
    Examples:
        >>> text1 = 'SQLQuery: SELECT * FROM table'
        >>> extract_sql(text1)
        'SELECT * FROM table'
        
        >>> text2 = '```sql\\nSELECT *\\nFROM table\\n```'
        >>> extract_sql(text2)
        'SELECT * FROM table'
    """
    if text is None:
        return None
        
    # 1. SQL 마크다운 코드 블록 제거
    markdown_pattern = r'```sql[ite]*\s*(.*?)\s*```'
    markdown_match = re.search(markdown_pattern, text, re.DOTALL)
    if markdown_match:
        text = markdown_match.group(1)
    
    # 2. SQLQuery: 패턴 처리
    sql_pattern = r'SQLQuery:\s*(.*?)(?=SQLQuery:|$)'
    sql_match = re.search(sql_pattern, text, re.DOTALL)
    if sql_match:
        text = sql_match.group(1)
    
    # 3. 쿼리 정리
    if text:
        # 줄바꿈을 공백으로 변경
        cleaned = text.replace('\n', ' ')
        # 연속된 공백을 하나로 변경
        cleaned = re.sub(r'\s+', ' ', cleaned)
        # 양쪽 공백 제거
        cleaned = cleaned.strip()
        # 마지막 세미콜론 제거 
        cleaned = re.sub(r';$', '', cleaned)
        
        return cleaned
        
    return None


# 테스트 (두 개의 생성된 모델의 출력을 사용)
gpt_cleaned_query = extract_sql(gpt_generated_sql)
gemini_cleaned_query = extract_sql(gemini_generated_sql)

print(gpt_cleaned_query)
print("-"*100)
print(gemini_cleaned_query)
print("-"*100)

SELECT "운용사", COUNT(*) AS "ETF_개수" FROM "ETFs" GROUP BY "운용사" ORDER BY "ETF_개수" DESC LIMIT 5
----------------------------------------------------------------------------------------------------
SELECT "운용사", COUNT(*) AS "ETF_개수" FROM ETFs GROUP BY "운용사" ORDER BY "ETF_개수" DESC LIMIT 5
----------------------------------------------------------------------------------------------------


In [17]:
# 쿼리 실행
gpt_result = db.run(gpt_cleaned_query)
gemini_result = db.run(gemini_cleaned_query) 

print(gpt_result)
print("-"*100)
print(gemini_result)
print("-"*100)

[('삼성자산운용', 202), ('미래에셋자산운용', 200), ('케이비자산운용', 118), ('한국투자신탁운용', 88), ('한화자산운용', 65)]
----------------------------------------------------------------------------------------------------
[('삼성자산운용', 202), ('미래에셋자산운용', 200), ('케이비자산운용', 118), ('한국투자신탁운용', 88), ('한화자산운용', 65)]
----------------------------------------------------------------------------------------------------


In [18]:
import time 

# 다른 질문들에 대해서 테스트
sample_questions = [
    "평균 총보수가 가장 높은 운용사는 어디인가요?",
    "순자산총액이 가장 큰 ETF는 무엇인가요?"
]

def compare_answer_question(question: str):
    try:
        gpt_generated_sql = gpt_sql.invoke({'question':question})
        gemini_generated_sql = gemini_sql.invoke({'question':question})
        
        gpt_cleaned_query = extract_sql(gpt_generated_sql)
        gemini_cleaned_query = extract_sql(gemini_generated_sql)
        
        gpt_result = db.run(gpt_cleaned_query) 
        gemini_result = db.run(gemini_cleaned_query)
        
        print(f"Question: {question}")
        print(f"Answer (GPT): {gpt_result}")
        print(f"Answer (Gemini): {gemini_result}")
        print("-"*100)
    except Exception as e:
        print(f"Error processing question: {question}")
        print(f"Error message: {str(e)}")
        print("-"*100)


for question in sample_questions:
    compare_answer_question(question)
    time.sleep(3)

Question: 평균 총보수가 가장 높은 운용사는 어디인가요?
Answer (GPT): [('에셋플러스자산운용', 0.98625), ('에셋플러스자산운용(주)', 0.9857142857142858), ('(주)타임폴리오자산운용', 0.8000000000000002), ('타임폴리오자산운용', 0.8), ('트러스톤자산운용(주)', 0.785)]
Answer (Gemini): [('에셋플러스자산운용',)]
----------------------------------------------------------------------------------------------------
Question: 순자산총액이 가장 큰 ETF는 무엇인가요?
Answer (GPT): [('KODEX CD금리액티브(합성)', 9591343573385.0)]
Answer (Gemini): [('KODEX CD금리액티브(합성)',)]
----------------------------------------------------------------------------------------------------


---

## [실습] **ETF 상세정보를 Text2SQL 구현**

- data/etf_info.csv 데이터를 SQLite 스키마를 정의하여 "ETFInfo" 테이블에 저장
- SQL 쿼리 체인을 실행하여 테스트 

In [19]:
etf_info.head(2)

항목,한글명,영문명,종목코드,상장일,펀드형태,기초지수명,추적배수,자산운용사,지정참가회사(AP),총보수(%),회계기간,과세유형,분배금 지급일,홈페이지,기초 시장,기초 자산,기본 정보,투자유의사항
내용,TIGER 엔비디아미국채커버드콜밸런스(합성),TIGER SYNTH-NVIDIA UST Cvd Call Balanced,0000D0,2024-12-17,수익증권형,KEDI 엔비디아미국채30년타겟커버드콜혼합지수(TR),일반 (1),미래에셋자산운용(주),"미래, 키움, 메리츠, 삼성",0.39,"매년 1월 1일부터 12월 31일까지. 단, 최초 회계기간은 최초설정일로부터 당해연...",배당소득세(보유기간과세),매월 마지막 영업일,www.tigeretf.com,(해외) (북미) (미국),(혼합자산) (주식+채권) / (-) (-) / (-),"- 이 ETF는 주로 채권 및 채권관련 집합투자증권을을 주된 투자대상자산으로 하며,...",- 이 ETF의 수익률은 보수 또는 비용 등 이 ETF의 순자산가치에 부의 영향을 ...
내용,KODEX 200,KODEX 200,069500,2002-10-14,수익증권형,코스피 200,일반 (1),삼성자산운용(주),NaN,0.15,매년 1월 1일부터 12월 31일까지,비과세,"매 1, 4, 7, 10월의 마지막 영업일 및 회계기간 종료일(종료일이 영업일이 아...",http://www.kodex.com,(국내) (코스피) (-),(주식) (시장대표) / (-) (-) / (-),1좌당 순자산가치의 변동률을 기초지수인 KOSPI200의 변동률과 유사하도록 투자신...,"규정에서 정하는 한도를 초과하여 추적오차가 발생하고 일정기간 이상 지속될 경우, 지..."


In [20]:
# 컬럼명 변경 - CSV에서 나온 실제 컬럼명과 맞추기
etf_info.columns = ['한글명', '영문명', '종목코드', '상장일', '펀드형태', '기초지수명', '추적배수', '자산운용사', 
       '지정참가회사', '총보수', '회계기간', '과세유형', '분배금지급일', '홈페이지', '기초시장',
       '기초자산', '기본정보', '투자유의사항']

In [21]:
# 자료형 확인 
etf_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 914 entries, 내용 to 내용
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   한글명     914 non-null    object
 1   영문명     914 non-null    object
 2   종목코드    914 non-null    object
 3   상장일     914 non-null    object
 4   펀드형태    914 non-null    object
 5   기초지수명   914 non-null    object
 6   추적배수    914 non-null    object
 7   자산운용사   914 non-null    object
 8   지정참가회사  890 non-null    object
 9   총보수     914 non-null    object
 10  회계기간    914 non-null    object
 11  과세유형    914 non-null    object
 12  분배금지급일  914 non-null    object
 13  홈페이지    914 non-null    object
 14  기초시장    914 non-null    object
 15  기초자산    914 non-null    object
 16  기본정보    914 non-null    object
 17  투자유의사항  914 non-null    object
dtypes: object(18)
memory usage: 135.7+ KB


In [22]:
### None 값 확인하고 적절한 값으로 변경

def create_etfs_info_table(conn, etf_data):
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS ETFsInfo")  # 테이블명: ETFsInfo -> 기존에 존재하면 삭제
    
    cursor.execute("""
    CREATE TABLE ETFsInfo (
        한글명 TEXT,
        영문명 TEXT,
        종목코드 TEXT PRIMARY KEY,
        상장일 TEXT,
        펀드형태 TEXT,
        기초지수명 TEXT,
        추적배수 TEXT,
        자산운용사 TEXT,
        지정참가회사 TEXT,
        총보수 TEXT,
        회계기간 TEXT,
        과세유형 TEXT,
        분배금지급일 TEXT,
        홈페이지 TEXT,
        기초시장 TEXT,
        기초자산 TEXT,
        기본정보 TEXT,
        투자유의사항 TEXT
    )
    """)
    

    # 데이터 삽입
    for _, row in etf_data.iterrows():
        try:
            cursor.execute("""
            INSERT INTO ETFsInfo VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                str(row['한글명']),
                str(row['영문명']),
                str(row['종목코드']),
                str(row['상장일']),
                str(row['펀드형태']),
                str(row['기초지수명']),
                str(row['추적배수']),
                str(row['자산운용사']),
                str(row['지정참가회사']),
                str(row['총보수']),
                str(row['회계기간']),
                str(row['과세유형']),
                str(row['분배금지급일']),
                str(row['홈페이지']),
                str(row['기초시장']),
                str(row['기초자산']),
                str(row['기본정보']),
                str(row['투자유의사항'])
            ))
        except Exception as e:
            print(f"오류 - 종목코드: {row['종목코드']}")
            print(f"상세: {str(e)}")
            continue


    conn.commit()
    cursor.execute("SELECT COUNT(*) FROM ETFsInfo")
    print(f"\n=== ETFsInfo 테이블 생성 완료 (총 {cursor.fetchone()[0]}개) ===")
    return conn


# 데이터베이스 연결 및 테이블 생성
conn = sqlite3.connect('etf_database.db')
conn = create_etfs_info_table(conn, etf_info)

# 데이터베이스 종료 
conn.close()


=== ETFsInfo 테이블 생성 완료 (총 914개) ===


In [23]:
# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri("sqlite:///etf_database.db")


# SQL Chain 설정
gpt_llm = ChatOpenAI(model="gpt-4o-mini")
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")

gpt_sql = create_sql_query_chain(llm=gpt_llm, db=db)
gemini_sql = create_sql_query_chain(llm=gemini_llm, db=db)

# 쿼리 실행
test_question = "추적배수가 일반 유형이고, 총보수가 0.1보다 작은 ETF 상품은 무엇인가요?"

gpt_generated_sql = gpt_sql.invoke({'question':test_question})
gemini_generated_sql = gemini_sql.invoke({'question':test_question})

print(f"Answer (GPT):\n{gpt_generated_sql}")
print("-"*100)
print(f"Answer (Gemini):\n{gemini_generated_sql}")
print("-"*100)

Answer (GPT):
SQLQuery: 
```sql
SELECT "종목코드", "종목명", "총보수"
FROM "ETFs"
WHERE "총보수" < 0.1 AND "종목코드" IN (
    SELECT "종목코드"
    FROM "ETFsInfo"
    WHERE "추적배수" = '일반 (1)'
)
LIMIT 5;
```
----------------------------------------------------------------------------------------------------
Answer (Gemini):
```sqlite
SELECT "한글명" FROM ETFsInfo WHERE "추적배수" = '일반 (1)' AND "총보수" < '0.1' LIMIT 5;
```
----------------------------------------------------------------------------------------------------


In [24]:
# 쿼리 실행
gpt_cleaned_query = extract_sql(gpt_generated_sql)
gemini_cleaned_query = extract_sql(gemini_generated_sql)

try:
    gpt_result = db.run(gpt_cleaned_query)
    print(f"""GPT Result: {gpt_result}""")
    print("-"*100)
except Exception as e:
    print(f"Error running GPT query: {str(e)}")

try:
    gemini_result = db.run(gemini_cleaned_query)
    print(f"""Gemini Result: {gemini_result}""")
    print("-"*100)
except Exception as e:
    print(f"Error running Gemini query: {str(e)}")

GPT Result: [('069660', 'KOSEF 200', 0.05), ('102110', 'TIGER 200', 0.05), ('105190', 'ACE 200', 0.09), ('114470', 'KOSEF 국고채3년', 0.05), ('122260', 'KOSEF 통안채1년', 0.05)]
----------------------------------------------------------------------------------------------------
Gemini Result: [('KOSEF 200',), ('TIGER 200',), ('ACE 200',), ('KOSEF 국고채3년',), ('KOSEF 통안채1년',)]
----------------------------------------------------------------------------------------------------
